In [1]:
import requests
import pandas as pd
from datetime import date
from io import BytesIO

from datetime import datetime
from dateutil.parser import parse
from tqdm import tqdm

import pymongo
from pymongo import MongoClient
from pymongo.server_api import ServerApi

In [2]:
# class to determine ganjil genap
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

In [3]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("KMTC")

In [4]:
# array for failed tracking
gagal=[]

In [5]:
hasil_akhir = []
for i, bls in enumerate(tqdm(bl_list)):
    try:
        # hitting the API
        url_ctr = "https://api.ekmtc.com/trans/trans/cargo-tracking/{}?dtKnd=BL&blNo={}".format(
            bls[4:], bls[4:])
        response_ctr = requests.request("GET", url_ctr)

        # getting container data. booking id needed for API URL
        ctr_data = response_ctr.json()
        list_of_container_number = []
        list_of_bookingid = []

        for j, ctr_info in enumerate(ctr_data['cntrList']):
            list_of_container_number.append(ctr_info["cntrNo"])
            list_of_bookingid.append(ctr_info["bkgNo"])

        for c, ctr in enumerate(list_of_container_number):
            url_milestone = "https://api.ekmtc.com/trans/trans/cargo-tracking/{}/detail?bkgNo={}&cntrNo={}&dtKnd=BL&strBkgNo={}".format(
                bls[4:], list_of_bookingid[c], ctr, list_of_bookingid[c])
            
            # hitting the API again for every container for container milestones
            response_milestone = requests.request("GET", url_milestone)

            # aqcuiring milestone datas for every container
            milstone_data = response_milestone.json()
            current_dict = {}
            dict_milestone = {}
            milestones = []
            key_mapping = {
                'blNo': 'BL Number',
                'cntrNo': 'Container Number',
                'etd': 'ETD',
                'eta': 'ETA',
                # 'GTOOB':'Gate Out Origin',
                # 'GTIOB':'Gate In Origin',
                'LDGOB': 'ATD',
                'DISIB': 'ATA',
                'GTOIB': 'Container Release',
                'GTIIB': 'Container Return'
            }

            current_dict.update({"Liners": "KMTC"})
            current_dict.update(
                {"From": milstone_data['trackingList'][-1]['plcNm'][:milstone_data['trackingList'][-1]['plcNm'].index(",")]})
            current_dict.update(
                {"To": milstone_data['trackingList'][-1]['podPortNm']})

            for key, label in key_mapping.items():
                if key in ctr_data['cntrList'][c]:
                    if key == "etd" or key == "eta":
                        current_dict[label] = datetime.strptime(
                            ctr_data['cntrList'][c][key][:8], "%Y%m%d").strftime("%Y-%m-%d")
                    else:
                        current_dict[label] = ctr_data['cntrList'][c][key]

            for b, milestone in enumerate(milstone_data['trackingList']):
                milestones.append(
                    milestone["cntrStsCd"] + milestone["cntrMvntCd"])
                milestones.append(milestone["mvntDt"])

            for m, milestone in enumerate(milestones):
                if is_date(milestone):
                    case_milestone = {
                        milestones[m-1]: datetime.strptime(milestone, "%Y%m%d").strftime("%Y-%m-%d")}
                    dict_milestone.update(case_milestone)

            for key, label in key_mapping.items():
                if key in dict_milestone:
                    current_dict[label] = dict_milestone[key]

            print(current_dict)
            hasil_akhir.append(current_dict)
    except Exception as e:
        print(e)
        print("{} GAGAL!!".format(bls))
        gagal.append(bls)


  0%|          | 0/21 [00:00<?, ?it/s]

{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4581147', 'Container Number': 'FCIU7270517', 'ETD': '2023-09-10', 'ETA': '2023-09-13', 'ATD': '2023-09-10', 'ATA': '2023-09-13'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4581147', 'Container Number': 'FFAU1044834', 'ETD': '2023-09-10', 'ETA': '2023-09-13', 'ATD': '2023-09-10', 'ATA': '2023-09-13'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4581147', 'Container Number': 'FFAU1058376', 'ETD': '2023-09-10', 'ETA': '2023-09-13', 'ATD': '2023-09-10', 'ATA': '2023-09-13'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4581147', 'Container Number': 'FFAU1064147', 'ETD': '2023-09-10', 'ETA': '2023-09-13', 'ATD': '2023-09-10', 'ATA': '2023-09-13'}


  5%|▍         | 1/21 [00:01<00:37,  1.89s/it]

{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4581147', 'Container Number': 'KMTU9361194', 'ETD': '2023-09-10', 'ETA': '2023-09-13', 'ATD': '2023-09-10', 'ATA': '2023-09-13'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4567181', 'Container Number': 'BMOU6207785', 'ETD': '2023-08-24', 'ETA': '2023-08-30', 'ATD': '2023-08-24', 'ATA': '2023-08-30', 'Container Release': '2023-09-12', 'Container Return': '2023-09-14'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4567181', 'Container Number': 'BMOU6932821', 'ETD': '2023-08-24', 'ETA': '2023-08-30', 'ATD': '2023-08-24', 'ATA': '2023-08-30', 'Container Release': '2023-09-11', 'Container Return': '2023-09-15'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4567181', 'Container Number': 'BMOU6935055', 'ETD': '2023-08-24', 'ETA': '2023-08-30', 'ATD': '2023-08-24', 'ATA': '2023-08-30', 'Container Release': '2023-09-10', 'Container Retu

 10%|▉         | 2/21 [00:04<00:44,  2.34s/it]

...

{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'SHEKOU', 'BL Number': 'JKT4577376', 'Container Number': 'TXGU5029453', 'ETD': '2023-09-02', 'ETA': '2023-09-11', 'ATD': '2023-09-02', 'ATA': '2023-09-11', 'Container Release': '2023-09-12', 'Container Return': '2023-09-13'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'XINGANG', 'BL Number': 'JKT4561058', 'Container Number': 'BEAU4739022', 'ETD': '2023-08-24', 'ETA': '2023-09-08', 'ATD': '2023-08-24', 'ATA': '2023-09-08', 'Container Release': '2023-09-11', 'Container Return': '2023-09-12'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'XINGANG', 'BL Number': 'JKT4561058', 'Container Number': 'FFAU1063578', 'ETD': '2023-08-24', 'ETA': '2023-09-08', 'ATD': '2023-08-24', 'ATA': '2023-09-08', 'Container Release': '2023-09-11', 'Container Return': '2023-09-12'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'XINGANG', 'BL Number': 'JKT4561058', 'Container Number': 'FFAU3974280', 'ETD': '2023-08-24', 'ETA': '2023-09-08', 'ATD': '2023-08-24', 'ATA': '202

 95%|█████████▌| 20/21 [00:38<00:01,  1.91s/it]

{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'XINGANG', 'BL Number': 'JKT4561058', 'Container Number': 'TXGU6054635', 'ETD': '2023-08-24', 'ETA': '2023-09-08', 'ATD': '2023-08-24', 'ATA': '2023-09-08', 'Container Release': '2023-09-12', 'Container Return': '2023-09-12'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'PHNOM PENH', 'BL Number': 'JKT4578948', 'Container Number': 'BEAU4868842', 'ETD': '2023-09-10', 'ETA': '2023-09-13', 'ATD': '2023-09-10'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'PHNOM PENH', 'BL Number': 'JKT4578948', 'Container Number': 'BMOU6918715', 'ETD': '2023-09-10', 'ETA': '2023-09-13', 'ATD': '2023-09-10'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'PHNOM PENH', 'BL Number': 'JKT4578948', 'Container Number': 'DFSU7692141', 'ETD': '2023-09-10', 'ETA': '2023-09-13', 'ATD': '2023-09-10'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'PHNOM PENH', 'BL Number': 'JKT4578948', 'Container Number': 'FCIU7123408', 'ETD': '2023-09-10', 'ETA': '2023-09-13', 'ATD': '2023-09-10'

100%|██████████| 21/21 [00:40<00:00,  1.94s/it]

{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'PHNOM PENH', 'BL Number': 'JKT4578948', 'Container Number': 'TXGU5029598', 'ETD': '2023-09-10', 'ETA': '2023-09-13', 'ATD': '2023-09-10'}


In [9]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(hasil_akhir)

Today's Collection Name ===>  all_tracking_Sep-18-2023
Today's Collection Has Been Made
Inserting Many Complete!!


In [7]:
# exporting to excel file
df = pd.DataFrame(hasil_akhir)
excel_file_path = 'export excel/KMTC.xlsx'
df.to_excel(excel_file_path, index=False)

In [7]:
gagal

[]